# [문제]
- 대화를 지속할 수 있도록 코드 구현
- 대화는 대문자 S를 입력하면 종료

In [18]:
# 1. 모듈 import
# from langchain_core.chat_history import InMemoryChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI


In [ ]:

# 2. .env 파일에서 환경변수 읽어오기
from dotenv import load_dotenv
load_dotenv()

# 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소 (type: dict)
store = {}

# 4. 함수 정의: 세션 ID에 따라 대화 히스로토리를 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

# 5. 프롬포트 템플릿 정의 
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

# 6. 쳇오픈 AI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_completion_tokens=150,
)

# 7. 체인 구성
chain = prompt | llm 

# 8. 체인에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

# 9. 체인 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': 'query'},
        config={'configurable': {'session_id': '1234'}}
    )

In [16]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='query', additional_kwargs={}, response_metadata={}), AIMessage(content='물론입니다! 이력서 작성을 위해 지원자의 경력, 학력, 기술, 자격증 등 구체적인 정보를 제공해 주시면 더욱 맞춤화된 이력서를 작성하는 데 도움이 됩니다. 아래와 같이 기본 정보를 공유해 주세요.\n\n1. 이름 및 연락처\n2. 지원하는 직무 및 목표\n3. 학력 (대학명, 전공, 졸업 연도 등)\n4. 경력 사항 (회사명, 직위, 담당 업무, 근무 기간)\n5. 주요 성과 또는 프로젝트\n6. 기술 및 자격증\n7. 기타 활동 또는 수상경력\n8. 자기소개서에 반영하고 싶은 핵심 메시지\n\n', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-93cef07d-f318-4bfe-a0ca-06e23ef9240c-0'), HumanMessage(content='query', additional_kwargs={}, response_metadata={}), AIMessage(content='이력서 작성을 위해 필요한 구체적인 정보나 내용을 제공해 주시면, 자연스럽고 매끄럽게 구성된 이력서를 작성해 드리겠습니다. 예를 들어, 지원자의 이름, 희망 직무, 학력, 경력, 기술 및 자격증 등을 알려주시면 됩니다. 상세 정보를 공유해 주세요!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='r

In [14]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

query
**************************************************
물론입니다! 이력서 작성을 위해 지원자의 경력, 학력, 기술, 자격증 등 구체적인 정보를 제공해 주시면 더욱 맞춤화된 이력서를 작성하는 데 도움이 됩니다. 아래와 같이 기본 정보를 공유해 주세요.

1. 이름 및 연락처
2. 지원하는 직무 및 목표
3. 학력 (대학명, 전공, 졸업 연도 등)
4. 경력 사항 (회사명, 직위, 담당 업무, 근무 기간)
5. 주요 성과 또는 프로젝트
6. 기술 및 자격증
7. 기타 활동 또는 수상경력
8. 자기소개서에 반영하고 싶은 핵심 메시지


**************************************************
query
**************************************************
이력서 작성을 위해 필요한 구체적인 정보나 내용을 제공해 주시면, 자연스럽고 매끄럽게 구성된 이력서를 작성해 드리겠습니다. 예를 들어, 지원자의 이름, 희망 직무, 학력, 경력, 기술 및 자격증 등을 알려주시면 됩니다. 상세 정보를 공유해 주세요!
**************************************************
